In [ ]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
import umap
import matplotlib
import mygene
%matplotlib inline
import pickle
import sklearn
import random
import scanpy as sc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
from xgboost import XGBClassifier
# import sentence_transformers
plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'
# plt.rcParams.update({
#     "text.usetex": False,
#     "font.family": "Helvetica"
# })
import matplotlib_inline
import scib_metrics
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
import openai
# remember to set your open AI API key!
openai.api_key = '' #replace it with your own API
np.random.seed(202310)
# use hnswlib for NN classification
try:
    import hnswlib
    hnswlib_imported = True
except ImportError:
    hnswlib_imported = False
    print("hnswlib not installed! We highly recommend installing it for fast similarity search.")
    print("To install it, run: pip install hnswlib")
from scipy.stats import mode

In [ ]:
# Here we consider steps to read different datasets.

# adata = sc.read("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/demo_train.h5ad")

# adata = sc.read("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/demo_test.h5ad")

# sampled_adata = sc.read_h5ad("../sample_aorta_data_updated.h5ad")
# sampled_adata = sampled_adata[np.where(sampled_adata.obs.celltype!='Unknown')[0]]
# adata = sampled_adata.copy()
# adata.obs['Celltype'] = adata.obs['celltype'].copy()

adata = sc.read("/gpfs/gibbs/pi/zhao/wl545/pbmc/datasets/3k_test.h5ad")
adata.obs['Celltype'] = adata.obs['label'].copy()

In [ ]:

def evaluate_nmi_ari(adata, label = 'scClassify', key='X_gpt3.5'):
    labels = np.array(list(adata.obs[label]))
    result1 = scib_metrics.nmi_ari_cluster_labels_leiden(adata.obsp['connectivities'], labels = labels, n_jobs = -1)
    result2 = scib_metrics.silhouette_label(adata.obsm[key], labels = labels, rescale=True, chunk_size=256)
    print(result1)
    print(result2)
    return result1, result2

In [ ]:

with open("/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/ensem_emb_gpt3.5all.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= list(adata.var.index)
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed_genept = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings:
        lookup_embed_genept[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1
        
with open("/gpfs/gibbs/pi/zhao/tl688/GenePT/data_embedding/GPT_3_5_gene_embeddings.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= list(adata.var.index)
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed_gpt35 = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings:
        lookup_embed_gpt35[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1        

In [ ]:
# we have different settings for embeddings

genePT_w_emebed =  (adata.X @ lookup_embed_genept) /len(gene_names) # GenePTWW

genePT_w_emebed = (adata.X @ lookup_embed_gpt35) /len(gene_names) # GPT 3.5 aa

genePT_w_emebed = adata.X / np.sum(adata.X, axis=1) [:,None]  @ lookup_embed_gpt35 # GPT 3.5 wa


lookup_embed = lookup_embed_genept + lookup_embed_gpt35 
genePT_w_emebed = adata.X / np.sum(adata.X, axis=1) [:,None]  @ lookup_embed # GenePT + GPT 3.5 wa

lookup_embed = np.concatenate([lookup_embed_genept, lookup_embed_gpt35], axis=1)
genePT_w_emebed = adata.X / np.sum(adata.X, axis=1) [:,None]  @ lookup_embed # GenePT || GPT 3.5 wa

print(f"Unable to match {count_missing} out of {len(gene_names)} genes in the GenePT-w embedding")
genePT_w_emebed_test = genePT_w_emebed

In [ ]:
# For cell type, considering aggregration embeddings
with open('ensem_emb_celltype.pickle', 'rb') as f:
    ct_name_getembedding = pickle.load(f)

lookup_embed_ct = np.zeros(shape=(len(adata),EMBED_DIM))
for i, gene in enumerate(adata.obs_names):
    lookup_embed_ct[i,:] = ct_name_getembedding[adata[gene].obs.Celltype.values[0]]

genePT_w_emebed_test += lookup_embed_ct

In [ ]:
# For PCA, we follow the pipeline from scanpy
sc.pp.scale(adata)
sc.tl.pca(adata)
genePT_w_emebed_test = adata.obsm['X_pca']

In [ ]:
adata.obsm['X_gpt3.5'] = genePT_w_emebed_test 
sc.pp.neighbors(adata, use_rep='X_gpt3.5')
evaluate_nmi_ari(adata, label = 'Celltype')

In [ ]:
sc.tl.umap(adata)
sc.pl.umap(adata, color='Celltype')